In [228]:
#Prophet Algorithm

In [299]:
!pip install pandas
!pip install matplotlib
!pip install prophet
!pip install scikit-learn
!pip install numpy

In [371]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import matplotlib.pyplot as plt

In [372]:
pd.options.display.max_columns=100

In [373]:
#Load The data
df_train = pd.read_csv('D:/MS_Web_and_Data Science_Sem_3/Data_Analysis_in_Cloud/Wallmart data/06-09-2023/train.csv')
df_features = pd.read_csv('D:/MS_Web_and_Data Science_Sem_3/Data_Analysis_in_Cloud/Wallmart data/06-09-2023/features.csv')

In [374]:
df_train.shape , df_features.shape

((421570, 5), (8190, 12))

In [375]:
df_features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


In [376]:
df_train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [377]:
df_train.isna().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [378]:
df_train.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
5,1,1,2010-03-12,21043.39,False
6,1,1,2010-03-19,22136.64,False
7,1,1,2010-03-26,26229.21,False
8,1,1,2010-04-02,57258.43,False
9,1,1,2010-04-09,42960.91,False


In [379]:
df_features.isna().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [380]:
# To convert Date to datetime
df_train["Date"] = pd.to_datetime(df_train["Date"])
df_features["Date"] = pd.to_datetime(df_features["Date"])

In [381]:
train_data = pd.merge(df_train, df_features, on=['Date','Store'], how='inner')

In [382]:
train_data.tail(30)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
421540,45,41,2012-10-26,1954.67,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421541,45,42,2012-10-26,4894.78,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421542,45,44,2012-10-26,3144.09,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421543,45,46,2012-10-26,13329.25,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421544,45,52,2012-10-26,1104.16,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421545,45,54,2012-10-26,23.92,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421546,45,55,2012-10-26,3458.17,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421547,45,56,2012-10-26,937.72,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421548,45,58,2012-10-26,275.00,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False
421549,45,59,2012-10-26,191.98,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False


In [383]:
train_data.isna().sum()

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
dtype: int64

In [384]:
drop_col = ["IsHoliday_y","MarkDown1", "MarkDown2", "MarkDown3","MarkDown4","MarkDown5","CPI","Temperature","Fuel_Price","Unemployment"]
train_data.drop(columns=drop_col, inplace=True)

In [385]:
train_data.head(10)

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
0,1,1,2010-02-05,24924.50,False
1,1,2,2010-02-05,50605.27,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
5,1,6,2010-02-05,5749.03,False
6,1,7,2010-02-05,21084.08,False
7,1,8,2010-02-05,40129.01,False
8,1,9,2010-02-05,16930.99,False
9,1,10,2010-02-05,30721.50,False


In [386]:
train_data.shape

(421570, 5)

In [387]:
duplicates = train_data[train_data.duplicated()]

# To Display the duplicate rows from dataset
if not duplicates.empty:
    print(duplicates)
else:
    print("No duplicate rows found.")

# Removing the duplicate rows
train_data = train_data.drop_duplicates()

No duplicate rows found.


In [388]:
# check for the outliers
forecasting_column = 'Weekly_Sales'

Data_1 = np.percentile(train_data[forecasting_column], 25)
Data_2 = np.percentile(train_data[forecasting_column], 75)


range_multiplier = 1.5 
Interquartile_Range = Data_2 - Data_1

lower_range = Data_1 - range_multiplier * Interquartile_Range
upper_range = Data_2 + range_multiplier * Interquartile_Range


detect_outliers = train_data[(train_data[forecasting_column] < lower_range) | (train_data[forecasting_column] > upper_range)]
 

In [389]:
detect_outliers

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
1,1,2,2010-02-05,50605.27,False
36,1,38,2010-02-05,115564.35,False
37,1,40,2010-02-05,66780.63,False
56,1,72,2010-02-05,98499.12,False
65,1,90,2010-02-05,76419.47,False
...,...,...,...,...,...
421497,45,92,2012-10-19,48108.38,False
421500,45,95,2012-10-19,48434.97,False
421538,45,38,2012-10-26,47799.07,False
421564,45,92,2012-10-26,54608.75,False


In [390]:
train_data = train_data[~train_data.index.isin(detect_outliers.index)]

In [391]:
train_data

,Store,Dept,Date,Weekly_Sales,IsHoliday_x
0,1,1,2010-02-05,24924.50,False
2,1,3,2010-02-05,13740.12,False
3,1,4,2010-02-05,39954.04,False
4,1,5,2010-02-05,32229.38,False
5,1,6,2010-02-05,5749.03,False
...,...,...,...,...,...
421563,45,91,2012-10-26,16330.84,False
421565,45,93,2012-10-26,2487.80,False
421566,45,94,2012-10-26,5203.31,False
421568,45,97,2012-10-26,6817.48,False


In [392]:
cols_to_drop = ["Store","Dept", "IsHoliday_x"]
train_data.drop(columns=cols_to_drop, inplace=True)

In [393]:
train_data

,Date,Weekly_Sales
0,2010-02-05,24924.50
2,2010-02-05,13740.12
3,2010-02-05,39954.04
4,2010-02-05,32229.38
5,2010-02-05,5749.03
...,...,...
421563,2012-10-26,16330.84
421565,2012-10-26,2487.80
421566,2012-10-26,5203.31
421568,2012-10-26,6817.48


In [394]:
train_data.rename(columns={'Date':'ds','Weekly_Sales':'y'}, inplace=True)

In [395]:
train_data

,ds,y
0,2010-02-05,24924.50
2,2010-02-05,13740.12
3,2010-02-05,39954.04
4,2010-02-05,32229.38
5,2010-02-05,5749.03
...,...,...
421563,2012-10-26,16330.84
421565,2012-10-26,2487.80
421566,2012-10-26,5203.31
421568,2012-10-26,6817.48


In [396]:
train_data.set_index('ds',inplace=True)

In [397]:
train_data

,y
ds,
2010-02-05,24924.50
2010-02-05,13740.12
2010-02-05,39954.04
2010-02-05,32229.38
2010-02-05,5749.03
...,...
2012-10-26,16330.84
2012-10-26,2487.80
2012-10-26,5203.31


In [398]:
#resample data on weekly basis
resampled_df = train_data.resample('W').mean()

In [399]:
resampled_df

,y
ds,
2010-02-07,10371.766994
2010-02-14,10945.257698
2010-02-21,10730.564159
2010-02-28,9917.680665
2010-03-07,10313.762879
...,...
2012-09-30,9994.956797
2012-10-07,10252.912027
2012-10-14,10409.566168


In [400]:
# To split the data in 80:20 ratio

train_data_size = int(len(resampled_df) * 0.80)
new_train_data = resampled_df.iloc[:train_data_size]
new_test_data = resampled_df.iloc[train_data_size:]

In [401]:
new_train_data = new_train_data.reset_index()

In [402]:
new_train_data.tail(5)

,ds,y
109,2012-03-11,10352.812416
110,2012-03-18,10519.416761
111,2012-03-25,10607.530871
112,2012-04-01,10481.808055
113,2012-04-08,11080.157876


In [403]:
new_test_data = new_test_data.reset_index()

In [404]:
new_test_data.tail(10)

,ds,y
19,2012-08-26,10764.371608
20,2012-09-02,10453.217431
21,2012-09-09,10448.079734
22,2012-09-16,9984.846597
23,2012-09-23,10020.394773
24,2012-09-30,9994.956797
25,2012-10-07,10252.912027
26,2012-10-14,10409.566168
27,2012-10-21,10416.818226
28,2012-10-28,10573.756051


In [405]:
len(new_test_data)

29

In [406]:
# windowing function

def create_windows(data, window_size, overlap):
    windows = []
    data_len = len(data)

    for starting_index in range(0, data_len - window_size + 1, overlap):
        last_idx = starting_index + window_size
        data_window = data.iloc[starting_index:last_idx]
        windows.append(data_window)

    return windows


In [407]:
window_size = 15
overlap = 7

In [408]:
train_windows = create_windows(new_train_data, window_size, overlap)

In [409]:
len(train_windows)

15

In [410]:
train_windows

[           ds             y
 0  2010-02-07  10371.766994
 1  2010-02-14  10945.257698
 2  2010-02-21  10730.564159
 3  2010-02-28   9917.680665
 4  2010-03-07  10313.762879
 5  2010-03-14  10389.785914
 6  2010-03-21  10640.457408
 7  2010-03-28  10344.508218
 8  2010-04-04  11097.259085
 9  2010-04-11  10869.529448
 10 2010-04-18  10316.213983
 11 2010-04-25  10428.156794
 12 2010-05-02  10277.435610
 13 2010-05-09  11052.336054
 14 2010-05-16  10738.875565,
            ds             y
 7  2010-03-28  10344.508218
 8  2010-04-04  11097.259085
 9  2010-04-11  10869.529448
 10 2010-04-18  10316.213983
 11 2010-04-25  10428.156794
 12 2010-05-02  10277.435610
 13 2010-05-09  11052.336054
 14 2010-05-16  10738.875565
 15 2010-05-23  10656.504325
 16 2010-05-30  11045.261173
 17 2010-06-06  11241.614306
 18 2010-06-13  11008.895745
 19 2010-06-20  10973.693482
 20 2010-06-27  10965.648747
 21 2010-07-04  11082.249751,
            ds             y
 14 2010-05-16  10738.875565
 15 2010-05-

In [411]:
test_windows = create_windows(new_test_data, window_size, overlap)

In [412]:
len(test_windows)

3

In [413]:
test_windows

[           ds             y
 0  2012-04-15  10386.491028
 1  2012-04-22  10269.392024
 2  2012-04-29  10116.420124
 3  2012-05-06  10337.295664
 4  2012-05-13  10685.687038
 5  2012-05-20  10871.964559
 6  2012-05-27  10833.467060
 7  2012-06-03  10833.009012
 8  2012-06-10  10865.805102
 9  2012-06-17  10936.950215
 10 2012-06-24  10982.104187
 11 2012-07-01  10767.328875
 12 2012-07-08  11004.578985
 13 2012-07-15  10533.845807
 14 2012-07-22  10608.572105,
            ds             y
 7  2012-06-03  10833.009012
 8  2012-06-10  10865.805102
 9  2012-06-17  10936.950215
 10 2012-06-24  10982.104187
 11 2012-07-01  10767.328875
 12 2012-07-08  11004.578985
 13 2012-07-15  10533.845807
 14 2012-07-22  10608.572105
 15 2012-07-29  10397.692331
 16 2012-08-05  10534.942897
 17 2012-08-12  10546.764854
 18 2012-08-19  10641.780791
 19 2012-08-26  10764.371608
 20 2012-09-02  10453.217431
 21 2012-09-09  10448.079734,
            ds             y
 14 2012-07-22  10608.572105
 15 2012-07-

In [414]:
df = pd.concat(train_windows, ignore_index=True).reset_index(drop=True)
df.columns = ["ds", "y"]

In [415]:
print(df.head())

          ds             y
0 2010-02-07  10371.766994
1 2010-02-14  10945.257698
2 2010-02-21  10730.564159
3 2010-02-28   9917.680665
4 2010-03-07  10313.762879


In [416]:
df , type(df) , df.head(31)

(            ds             y
 0   2010-02-07  10371.766994
 1   2010-02-14  10945.257698
 2   2010-02-21  10730.564159
 3   2010-02-28   9917.680665
 4   2010-03-07  10313.762879
 ..         ...           ...
 220 2012-03-04  10037.057146
 221 2012-03-11  10352.812416
 222 2012-03-18  10519.416761
 223 2012-03-25  10607.530871
 224 2012-04-01  10481.808055
 
 [225 rows x 2 columns],
 pandas.core.frame.DataFrame,
            ds             y
 0  2010-02-07  10371.766994
 1  2010-02-14  10945.257698
 2  2010-02-21  10730.564159
 3  2010-02-28   9917.680665
 4  2010-03-07  10313.762879
 5  2010-03-14  10389.785914
 6  2010-03-21  10640.457408
 7  2010-03-28  10344.508218
 8  2010-04-04  11097.259085
 9  2010-04-11  10869.529448
 10 2010-04-18  10316.213983
 11 2010-04-25  10428.156794
 12 2010-05-02  10277.435610
 13 2010-05-09  11052.336054
 14 2010-05-16  10738.875565
 15 2010-03-28  10344.508218
 16 2010-04-04  11097.259085
 17 2010-04-11  10869.529448
 18 2010-04-18  10316.213983
 19

In [417]:
model = Prophet(
    # seasonality_mode='multiplicative',
    # holidays=None
)

In [418]:
model.fit(df)

02:50:20 - cmdstanpy - INFO - Chain [1] start processing
02:50:21 - cmdstanpy - INFO - Chain [1] done processing


In [421]:
# Create a dataframe for future predictions
future = model.make_future_dataframe(periods=len(test_windows))

In [422]:
# code to forecast predictions
forecast = model.predict(future)
forecast.tail(10)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
106,2012-02-19,10496.846238,9977.467534,10752.883852,10496.846238,10496.846238,-119.166328,-119.166328,-119.166328,-119.166328,-119.166328,-119.166328,0.0,0.0,0.0,10377.679909
107,2012-02-26,10497.199036,9731.603194,10565.955356,10497.199036,10497.199036,-335.695701,-335.695701,-335.695701,-335.695701,-335.695701,-335.695701,0.0,0.0,0.0,10161.503335
108,2012-03-04,10497.551834,9645.849765,10452.789841,10497.551834,10497.551834,-440.266514,-440.266514,-440.266514,-440.266514,-440.266514,-440.266514,0.0,0.0,0.0,10057.285320
109,2012-03-11,10497.904632,9668.360424,10541.431883,10497.904632,10497.904632,-374.976738,-374.976738,-374.976738,-374.976738,-374.976738,-374.976738,0.0,0.0,0.0,10122.927894
110,2012-03-18,10498.257431,9843.935069,10612.806600,10498.257431,10498.257431,-261.299698,-261.299698,-261.299698,-261.299698,-261.299698,-261.299698,0.0,0.0,0.0,10236.957733
111,2012-03-25,10498.610229,9919.525198,10693.852482,10498.610229,10498.610229,-195.280589,-195.280589,-195.280589,-195.280589,-195.280589,-195.280589,0.0,0.0,0.0,10303.329640
112,2012-04-01,10498.963027,9904.958050,10750.658180,10498.963027,10498.963027,-158.551962,-158.551962,-158.551962,-158.551962,-158.551962,-158.551962,0.0,0.0,0.0,10340.411065
113,2012-04-02,10499.013427,9928.307731,10753.789309,10499.013427,10499.013427,-152.262454,-152.262454,-152.262454,-152.262454,-152.262454,-152.262454,0.0,0.0,0.0,10346.750973
114,2012-04-03,10499.063827,9935.933255,10762.531552,10499.063827,10499.063827,-145.391586,-145.391586,-145.391586,-145.391586,-145.391586,-145.391586,0.0,0.0,0.0,10353.672241
115,2012-04-04,10499.114227,9945.800170,10772.614840,10499.114227,10499.114227,-137.924183,-137.924183,-137.924183,-137.924183,-137.924183,-137.924183,0.0,0.0,0.0,10361.190043


In [423]:
new_df = test_windows[1]
actual_values = new_df['y']

In [424]:
predicted_value = forecast.loc[:len(actual_values)-1, 'yhat']

In [425]:
len(predicted_value)

15

In [426]:
len(actual_values)

15

In [427]:
rmse_value = sqrt(mean_squared_error(actual_values, predicted_value))

In [428]:
rmse_value

312.41437677507895

In [429]:
actual_values = actual_values.tolist()
predicted_value = predicted_value.tolist()

In [430]:
data_2 = pd.DataFrame()

In [431]:
data_2['Actual Sales'] = actual_values
data_2['Predicted Sales'] = predicted_value

In [432]:
data_2

,Actual Sales,Predicted Sales
0,10833.009012,10474.405458
1,10865.805102,10702.305496
2,10936.950215,10556.023438
3,10982.104187,10352.095335
4,10767.328875,10311.094648
5,11004.578985,10406.718960
6,10533.845807,10504.487161
7,10608.572105,10549.814329
8,10397.692331,10586.368065
9,10534.942897,10644.294677


In [433]:
print("Root Mean Squared Error for the Prophet Model:",rmse_value)

Root Mean Squared Error for the Prophet Model: 312.41437677507895
